In [13]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Activation, Dense
# from tensorflow.keras import Flatten

In [15]:
img_height,img_width=180,180
batch_size=32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "train",
  validation_split=0.1,
  subset="training",
  seed=123,
  label_mode='categorical' ,
  # horizontal_flip,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 808 files belonging to 2 classes.
Using 728 files for training.


In [17]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "val",
  validation_split=0.1,
  subset="validation",
  seed=123,
  label_mode='categorical' ,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 139 files belonging to 2 classes.
Using 13 files for validation.


In [18]:
class_names=train_ds.class_names
print(class_names)

['not_ripe', 'ripe']


In [19]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=6,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.Flatten())

resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(6, activation='softmax'))


In [20]:
resnet_model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=35)

Epoch 1/35


ValueError: in user code:

    File "C:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Acer\AppData

In [22]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
test_image=tf.keras.utils.load_img('test.jpg',target_size=(180,180))
test_image=tf.keras.utils.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
pred=resnet_model.predict(test_image)